In [36]:
from gensim.models import KeyedVectors
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import re
import numpy as np
from torch.utils.data import DataLoader, Dataset
import spacy

In [37]:
word2vec_1 = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [63]:
data = pd.read_csv("merged_training.csv")
print(spacy.__version__)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
label_mapping = {"joy": 0, "sadness": 1, "anger": 2, "fear": 3, "love": 4, "surprise": 5}
data["label_encoded"] = data["label"].map(label_mapping)
important_stopwords = {
    "not", "no", "never", "none", "nowhere", "nothing", "neither", "nor", 
    "n't", "cannot", "without", 
    "very", "more", "most", "least", "less", "much", "many", "quite", "so",
    "such", "just", "too", "enough", "almost", "rather", "even",
    "if", "unless", "though", "although", "while", "whereas", 
    "few", "little", "hardly", "scarcely", "seldom",
    "before", "after", "until", "since", "when", "whenever", "once", 
    "against", "despite", "because", "besides", "however", "otherwise", "yet"
}

def tokenizer(text):
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc 
        if token.is_alpha and (not token.is_stop or token.text in important_stopwords)
    ]
    return tokens
def encode(sentence):
    vectors = [word2vec_1[word] for word in tokenizer(sentence) if word in word2vec_1]  
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_1.vector_size)
input_tensor = torch.tensor(encode("i don't love you"), dtype=torch.float).unsqueeze(0)
print(input_tensor.shape)

3.8.4
torch.Size([1, 300])


In [47]:
class TextDataset(Dataset):
    def __init__(self, data):
        self.texts = [torch.tensor(encode(sentence), dtype=torch.float) for sentence in data["text"]]
        self.labels = torch.tensor(data["label_encoded"].values, dtype=torch.int64)
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]
dataset = TextDataset(data)
dataloader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

In [65]:
torch.save(dataset, "dataset_emotion.pt")

In [76]:
class TextClassification(nn.Module):
    def __init__(self, input = 300, hidden = 128, output = 32, classes = 6):
        super(TextClassification, self).__init__()
        self.fc1 = nn.Linear(input, hidden)
        self.bn1 = nn.BatchNorm1d(hidden)
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(hidden, output)
        self.bn2 = nn.BatchNorm1d(output)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(output, classes)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x
model_test = TextClassification()
for name, param in model_test.named_parameters():
    print(f"{name}: {param.numel()} parameters")
total_params = sum(p.numel() for p in model_test.parameters())
print(f"Total parameters: {total_params}")

fc1.weight: 38400 parameters
fc1.bias: 128 parameters
bn1.weight: 128 parameters
bn1.bias: 128 parameters
fc2.weight: 4096 parameters
fc2.bias: 32 parameters
bn2.weight: 32 parameters
bn2.bias: 32 parameters
fc3.weight: 192 parameters
fc3.bias: 6 parameters
Total parameters: 43174


In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 50
for epoch in range(epochs):
    total_loss = 0.0
    total_samples = 0
    correct = 0
    for text, label in dataloader:
        text = text.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        out = model(text)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = torch.argmax(out, dim=1)
        correct+= torch.eq(pred, label).sum().item()
        total_samples += label.size(0)
    acc = 100*correct/total_samples
    avg_loss = total_loss/len(dataloader)
    print(f"epoch {epoch+1} loss = {avg_loss:.4f} accuracy = {acc:.4f}")

epoch 1 loss = 0.7810 accuracy = 71.0762
epoch 2 loss = 0.6790 accuracy = 74.7589
epoch 3 loss = 0.6515 accuracy = 75.6891
epoch 4 loss = 0.6363 accuracy = 76.1764
epoch 5 loss = 0.6234 accuracy = 76.6020
epoch 6 loss = 0.6167 accuracy = 76.8493
epoch 7 loss = 0.6114 accuracy = 77.0223
epoch 8 loss = 0.6057 accuracy = 77.2219
epoch 9 loss = 0.6002 accuracy = 77.3345
epoch 10 loss = 0.5981 accuracy = 77.5132
epoch 11 loss = 0.5963 accuracy = 77.5142
epoch 12 loss = 0.5928 accuracy = 77.6727
epoch 13 loss = 0.5930 accuracy = 77.7474
epoch 14 loss = 0.5893 accuracy = 77.8253
epoch 15 loss = 0.5871 accuracy = 77.8601
epoch 16 loss = 0.5848 accuracy = 77.9059
epoch 17 loss = 0.5818 accuracy = 77.9580
epoch 18 loss = 0.5830 accuracy = 77.9503
epoch 19 loss = 0.5820 accuracy = 77.9957
epoch 20 loss = 0.5810 accuracy = 78.0249
epoch 21 loss = 0.5783 accuracy = 78.1252
epoch 22 loss = 0.5782 accuracy = 78.1723
epoch 23 loss = 0.5788 accuracy = 78.1080
epoch 24 loss = 0.5755 accuracy = 78.2332
e

In [79]:
def predict(text, model, device):
    model.eval()
    input_tensor = torch.tensor(encode(text), dtype=torch.float).to(device)
    input_tensor = input_tensor.unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
    predicted_label = torch.argmax(output, dim=1).item()
    return predicted_label
label_arr = ["joy", "sadness", "anger", "fear", "love", "surprise"]
test_sentences = [
    # Joy (Happy)
    "Today is such a wonderful day!", 
    "I feel so happy to see my old friend again.",
    "Winning this competition makes me ecstatic!",
    "This food is so delicious, I love it!",
    "I received a gift, I'm so happy!",
    "The sun is shining, and I feel amazing!",
    "My best friend just got married, I'm so happy for them!",
    "I finally achieved my goal, what a great feeling!",
    "Listening to my favorite song makes my day.",
    "I got promoted today, I'm so excited!",
    "The laughter of children is so heartwarming.",
    "I just found out I won the lottery!",
    "I love spending time with my family, it brings me so much joy.",
    "This vacation is everything I dreamed of!",
    "My dog greeted me so happily when I got home!",
    "Getting a hug from someone I love feels amazing.",
    "I baked a cake, and it turned out perfect!",
    "I finally learned how to swim, I'm so proud of myself!",
    "Watching the sunset is so peaceful and beautiful.",
    "Today is my birthday, and I'm surrounded by love.",

    # Sadness (Sad)
    "I feel so lonely and lost.",
    "Today is a terrible day for me.",
    "I miss the old days, it makes me sad.",
    "It's heartbreaking to say goodbye to someone you love.",
    "I failed my exam, I'm really upset.",
    "Today feels so empty and meaningless.",
    "I miss my childhood, those were simpler times.",
    "Nothing seems to be going right lately.",
    "It hurts when someone you love ignores you.",
    "Losing a friend is one of the worst feelings.",
    "I feel like I have no one to talk to.",
    "I failed again, maybe I'm just not good enough.",
    "I miss the person I used to be.",
    "Some days, I just want to disappear.",
    "The world feels so cold and lonely sometimes.",
    "I just got rejected, and it really hurts.",
    "I feel like I'm carrying the weight of the world.",
    "It's painful when people forget about you.",
    "I cried myself to sleep last night.",
    "Seeing old photos makes me feel so nostalgic.",

    # Anger (Mad)
    "I'm so angry that I was lied to!",
    "Stop bothering me!",
    "I really angry.",
    "Why would you do that? I'm really mad!",
    "I've warned them, but they still will not listen!",
    "I can't believe they lied to me again!",
    "Stop wasting my time!",
    "I'm sick and tired of this nonsense!",
    "How many times do I have to repeat myself?!",
    "This is so unfair, I'm really mad!",
    "I just want to punch something right now!",
    "That was completely uncalled for!",
    "I can't stand being ignored!",
    "People need to learn some respect!",
    "I've had enough of this nonsense!",
    "I can't believe they betrayed me like that!",
    "I hate when people don't keep their promises!",
    "That was the last straw, I'm done!",
    "I'm not going to tolerate this anymore!",
    "Why do people always take me for granted?!",

    # Fear (Scared)
    "I'm nervous about my exam tomorrow.",
    "It's so dark in here, I'm scared.",
    "I'm terrified of the dark.",
    "That sound was really terrifying, I hate it!",
    "I'm afraid of losing my loved ones.",
    "I have a bad feeling about this.",
    "Walking alone at night makes me anxious.",
    "I keep hearing strange noises in my house.",
    "I don't want to be alone right now.",
    "This horror movie is too scary for me.",
    "My hands are shaking, I'm so nervous.",
    "I'm afraid of failing and disappointing everyone.",
    "That nightmare was absolutely terrifying.",
    "I feel like someone is watching me.",
    "The thought of getting lost in the forest scares me.",
    "I hate feeling helpless in scary situations.",
    "I'm not sure I can handle this, I'm too scared.",
    "I feel uneasy when I'm in crowded places.",
    "Thunderstorms make me really nervous.",
    "I can't move, I'm frozen in fear.",

    # Love (Loving)
    "I love you with all my heart.",
    "Every moment with you is precious to me.",
    "You mean the world to me.",
    "I can't imagine my life without you.",
    "Holding your hand makes me feel safe and loved.",
    "I feel so lucky to have you in my life.",
    "Your smile brightens my day.",
    "No matter what happens, I will always love you.",
    "Being with you is the best part of my day.",
    "I cherish every memory we make together.",
    "You are my soulmate, my everything.",
    "I fall in love with you more every day.",
    "Your love makes me feel complete.",
    "I can't stop thinking about you.",
    "You are the best thing that ever happened to me.",
    "You are my forever and always.",
    "I feel so connected to you, even when we're apart.",
    "Every love song reminds me of you.",
    "I just want to hold you close and never let go.",
    "You are my heart and soul.",

    # Surprise (Surprised)
    "Wow, I didn't see that coming!",
    "This is the best surprise ever!",
    "I can't believe my eyes!",
    "You really caught me off guard with that news!",
    "I'm speechless, this is unbelievable!",
    "I had no idea this was going to happen!",
    "That's amazing, I'm totally shocked!",
    "I never expected this at all!",
    "This is beyond what I imagined!",
    "I can't believe you did this for me!",
    "This is such a wonderful surprise!",
    "No way! Are you serious?!",
    "I'm in complete shock right now.",
    "I can't believe this is happening!",
    "This is so unexpected, but I love it!",
    "I never thought I would win!",
    "You totally surprised me with that!",
    "I'm so amazed by this news!",
    "This is the most unexpected thing ever!",
    "You just made my day with this surprise!"
]
dem = 0
acc = 0
for text in test_sentences:
    dem+=1
    pred = predict(text, model, device)
    print(label_arr[pred])
    if (dem == 20):
        print("----")
        dem = 0

joy
joy
joy
joy
joy
surprise
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
----
sadness
sadness
sadness
sadness
sadness
sadness
sadness
sadness
sadness
sadness
love
sadness
sadness
sadness
anger
sadness
joy
sadness
sadness
love
----
anger
anger
anger
anger
anger
anger
sadness
anger
sadness
anger
anger
anger
sadness
joy
joy
sadness
anger
anger
anger
joy
----
fear
fear
fear
fear
fear
sadness
fear
fear
joy
fear
fear
sadness
fear
sadness
sadness
fear
fear
fear
fear
fear
----
love
joy
joy
sadness
love
joy
sadness
joy
joy
joy
joy
love
joy
joy
joy
sadness
sadness
love
sadness
love
----
surprise
surprise
joy
anger
surprise
anger
surprise
joy
love
joy
surprise
joy
surprise
anger
sadness
sadness
surprise
surprise
sadness
surprise
----
